# [xTuring](https://github.com/stochasticai/xturing) - LLaMA INT4 efficient fine-tuning tutorial

This tutorial aims to show how easy it is to perform fine-tuning with xTuring. This notebook shows how to fine-tune LLaMA 7B model on GPU which has limited memory, it requires only 6GB VRAM

## 1. Install the `xTuring` library

In [ ]:
!pip install xturing --upgrade

In [7]:
import subprocess
from pathlib import Path

def pull_docker_image(image):
    cmd = ["docker", "pull", image]
    subprocess.run(cmd, check=True)


def run_docker_container(image, port_mapping, env_vars=None, gpus=None, volumes=None):
    cmd = ["docker", "container", "run"]

    if env_vars is None:
        env_vars = {}

    if volumes is None:
        volumes = {}

    if gpus is not None:
        cmd.extend(["--gpus", gpus])

    for key, value in env_vars.items():
        cmd.extend(["-e", f"{key}={value}"])

    for local_path, container_path in volumes.items():
        cmd.extend(["-v", f"{str(Path(local_path).resolve())}:{container_path}"])

    cmd.extend(["-p", port_mapping, image])

    subprocess.run(cmd)

## Load and run docker image

1. Install Docker on your machine if you haven't already. You can follow the [official Docker documentation](https://docs.docker.com/engine/install/) for installation instructions.
2. Install NVIDIA Container Toolkit
    ```bash
    sudo apt-get install -y nvidia-docker2
    ```
3. Run the Docker daemon
    ```bash
    sudo systemctl start docker
    ```


In [ ]:
image = "public.ecr.aws/t8g5g2q5/xturing:int4_finetuning"
port_mapping = "5000:5000"
env_vars = {
    "WANDB_MODE": "dryrun",
    "MICRO_BATCH_SIZE": "1", # change this to increase your micro batch size
}
# if you want to log results to wandb, set the following env var
# env_vars = {
#     "WANDB_API_KEY": "<your_wandb_api_key>",
#     "WANDB_PROJECT": "your_project_name",
#     "WANDB_ENTITY": "your_entity_name",
#     # Add more environment variables as needed
# }
volumes = {
    # "<where to save model>": "/model",
    "../llama/alpaca_data": "/data", # change this to your data path if you want
}
gpus = "all"

pull_docker_image(image)

run_docker_container(image, port_mapping, env_vars, gpus, volumes)

If you prefer it, you can run the Docker container from the CLI using the following command:

```bash
docker run -p 5000:5000 --gpus all -e WANDB_MODE=dryrun -e MICRO_BATCH_SIZE=1 -v /absolute/path/to/alpaca/data:/data public.ecr.aws/t8g5g2q5/xturing:int4_finetuning
```